In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Lasso, Ridge, RidgeCV, ElasticNet, ElasticNetCV, MultiTaskElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE

In [7]:
acled_df = pd.read_csv('../data/acled/south_sudan.csv') 
# acled_df = acled_df.drop(['SOURCE', 'SOURCE_SCALE', 'TAGS', 'EVENT_DATE', 'YEAR', 'GEO_PRECISION'], axis=1)

data = acled_df
X = data.drop('FATALITIES', axis=1)


# vector = np.array(data['FATALITIES'])
# y = np.where(vector > 0, np.log(vector), 0)
y = data['FATALITIES']

In [17]:
ucdp_df = pd.read_csv("../data/ucdp/south_sudan.csv").drop('low', axis=1) ## 1669 entries, 48 cols
ucdp_df.columns

Index(['id', 'relid', 'year', 'active_year', 'code_status', 'type_of_violence',
       'conflict_dset_id', 'conflict_new_id', 'conflict_name', 'dyad_dset_id',
       'dyad_new_id', 'dyad_name', 'side_a_dset_id', 'side_a_new_id', 'side_a',
       'side_b_dset_id', 'side_b_new_id', 'side_b', 'number_of_sources',
       'source_article', 'source_office', 'source_date', 'source_headline',
       'source_original', 'where_prec', 'where_coordinates',
       'where_description', 'adm_1', 'adm_2', 'latitude', 'longitude',
       'geom_wkt', 'priogrid_gid', 'country', 'country_id', 'region',
       'event_clarity', 'date_prec', 'date_start', 'date_end', 'deaths_a',
       'deaths_b', 'deaths_civilians', 'deaths_unknown', 'best', 'high',
       'gwnoa', 'gwnob'],
      dtype='object')

In [19]:
columns_to_keep = ['relid', 'year', 'active_year', 'best', 'conflict_new_id', 'conflict_name', 'dyad_dset_id',
       'dyad_new_id', 'dyad_name', 'where_prec', 'where_coordinates',
       'where_description', 'adm_1', 'adm_2', 'latitude', 'longitude',
       'geom_wkt', 'priogrid_gid', 'country', 'country_id', 'region', 'date_prec', 'date_start']
ucdp_df = ucdp_df[columns_to_keep]

ucdp_df.dropna(inplace=True)

data = ucdp_df
X = data.drop('best', axis=1)


# vector = np.array(data['FATALITIES'])
# y = np.where(vector > 0, np.log(vector), 0)
y = data['best']

In [20]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a column transformer to apply different preprocessing to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X = preprocessor.fit_transform(X)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(max_depth=3, random_state=0)
model.fit(X_train, y_train)
model.score(X_test, y_test)

-0.011351196878842451

In [22]:
y_pred = model.predict(X_test)
yp=y_pred

mse = mean_squared_error(y_test, yp)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 2399.5140980359247
